## Automatic Cardiology Report Generator
The following notebook generates medical reports for a cardiology practice. The pipeline functions as follows:

### Patient Information Input

- **Patient ID Entry:** Collects and displays patient's ID code.
- **Demographic Information:** Gathers age, sex, and ethnicity.
- **Blood Pressure Input:** Records systolic and diastolic blood pressure.
- **Auscultation and Test Results:** Allows selection of breath sounds, heart sounds, and other test results.

### Medical Report Generation

- **generate_medical_report Function:** Concatenates patient data and uses OpenAI API to generate medical reports.

### Text Quality Assessment

- **calculate_text_quality_BIOBERT Function:** Uses Bio_ClinicalBERT model to assess text quality.
- **calculate_text_quality_ROBERTA Function:** Utilizes XLM-Roberta model for text quality assessment.
- **calculate_text_quality_GPT Function:** Assess text quality using ChatGPT 3.4.

In [1]:
# install dependencies

!pip install openai
import openai
import os
!pip install transformers
from transformers import (
    XLMRobertaForSequenceClassification,
    XLMRobertaTokenizer,
    BertForSequenceClassification,
    BertTokenizer
)
import torch
import time
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.9 MB/s 

## Patient Information Input

This section describes the functions responsible for collecting essential information about the patient's cardiac status.

### 1. **Patient ID Entry:**
Prompts the user to enter the patient's ID code and displays the entered patient code.

### 2. **Demographic Information: Age, Sex, and Ethnicity Input:**
Prompts the user to enter the patient's age, sex, and ethnicity. These details are crucial for understanding the patient's cardiac health within a demographic context.

### 3. **Blood Pressure Input:**
Prompts the user to enter systolic and diastolic blood pressure values. If valid numerical inputs are provided, it displays the registered blood pressure of the patient.

### 4. **Auscultation and Test Results:**
Allows the user to select auscultated breath sounds, heart sounds, and other relevant test results. These details are important for cardiac diagnosis. The specific implementations for auscultated breath sounds, heart sounds, and other test results are provided in the code.

In [5]:
def enter_patient_id():
    patient_id = input("Enter patient ID code: ")
    p_ID = ("The patient code is:", patient_id)
    return p_ID

patient_id = enter_patient_id()


def age_d(value):
    message = f"The patient is {value} years old."
    #print(message)
    return message

value = input("Enter the patient'age: ")

try:
    value = int(value)
    age = age_d(value)
except ValueError:
    print("Invalid input! Please enter a numerical value.")



def select_gender(item):
    print(f"Selected gender: {item}")
    return item

items = ["Female", "Male", 'Trans-female', 'Trans-male']

print("Select the patient's gender:")
for i, item in enumerate(items, start=1):
    print(f"{i}. {item}")

choice = input("Enter your choice: ")
choice = int(choice)

selected_item = None

if 1 <= choice <= len(items):
    selected_gender = select_gender(items[choice - 1])
else:
    print("Invalid choice!")


# Use the selected_item variable for further processing if needed
if selected_gender:
    # Perform actions with the selected item
    gender = f"The gender of the patient is : {selected_gender}."
else:
    print("No valid item selected.")




def BP(value1, value2):
    message = f"The registered blood pressure of the patient is systolic {value1} and diastolic {value2} mmHg."
    #print(message)
    return message

value1 = input("Enter systolic BP: ")
value2 = input("Enter diastolic BP: ")


try:
    value1, value2 = float(value1), float(value2)
    BP_reg = BP(value1, value2)
except ValueError:
    print("Invalid input! Please enter a numerical value.")


def select_breath(item, intensity=None):
    if intensity:
         breath_aus = (f"The ausculted breath sounds were {item} and the intensity of the sound was {intensity}.")
         return breath_aus
        # Code to play the selected breath sound with the specified intensity
    else:
        breath_aus = (f"The ausculted breath sounds were {item}.")
        return breath_aus
        # Code to play the selected breath sound (without intensity)

items = ["Crackles (rales)", "Wheezes", "Ronchi", "Normal"]
intensities = ["mild", "medium", "intense"]

print("Select ausculted breath sound:")
for i, item in enumerate(items, start=1):
    print(f"{i}. {item}")

choice = input("Enter your choice: ")
choice = int(choice)

if 1 <= choice <= len(items):
    selected_item = items[choice - 1]

    if selected_item == "Normal":
        selected_breath = select_breath(selected_item)  # No intensity for "Normal"
    else:
        print("Select intensity:")
        for i, intensity in enumerate(intensities, start=1):
            print(f"{i}. {intensity}")

        intensity_choice = input("Enter intensity choice: ")
        intensity_choice = int(intensity_choice)

        if 1 <= intensity_choice <= len(intensities):
            selected_intensity = intensities[intensity_choice - 1]
            selected_breath = select_breath(selected_item, selected_intensity)
        else:
            print("Invalid intensity choice!")
else:
    print("Invalid choice!")


def select_heart(item, intensity=None):
    if intensity:
         breath_aus = (f"The ausculted heart sounds were {item} and the intensity of the sound was {intensity}.")
         return breath_aus
        # Code to play the selected breath sound with the specified intensity
    else:
        breath_aus = (f"The ausculted heart sounds were {item}.")
        return breath_aus
        # Code to play the selected breath sound (without intensity)

items = ["Murmurs", "Gallops", "Normal"]
intensities = ["mild", "medium", "intense"]

print("Select ausculted heart sounds:")
for i, item in enumerate(items, start=1):
    print(f"{i}. {item}")

choice = input("Enter your choice: ")
choice = int(choice)

if 1 <= choice <= len(items):
    selected_item = items[choice - 1]

    if selected_item == "Normal":
        selected_heart = select_heart(selected_item)  # No intensity for "Normal"
    else:
        print("Select intensity:")
        for i, intensity in enumerate(intensities, start=1):
            print(f"{i}. {intensity}")

        intensity_choice = input("Enter intensity choice: ")
        intensity_choice = int(intensity_choice)

        if 1 <= intensity_choice <= len(intensities):
            selected_intensity = intensities[intensity_choice - 1]
            selected_heart = select_heart(selected_item, selected_intensity)
        else:
            print("Invalid intensity choice!")
else:
    print("Invalid choice!")


def select_other(item, intensity=None):
    if intensity:
         breath_aus = (f"There was also a {item} sound in the auscultation and the intensity of the sound was {intensity}.")
         return breath_aus
        # Code to play the selected breath sound with the specified intensity
    else:
        breath_aus = (f" There was no other ausculted sound such us friction rubs or adventitious sound.")
        return breath_aus
        # Code to play the selected breath sound (without intensity)

items = ["friction rubs", "Gallops", "Normal"]
intensities = ["mild", "medium", "intense"]

print("Select other ausculted sounds:")
for i, item in enumerate(items, start=1):
    print(f"{i}. {item}")

choice = input("Enter your choice: ")
choice = int(choice)

if 1 <= choice <= len(items):
    selected_item = items[choice - 1]

    if selected_item == "Normal":
        selected_other = select_other(selected_item)  # No intensity for "Normal"
    else:
        print("Select intensity:")
        for i, intensity in enumerate(intensities, start=1):
            print(f"{i}. {intensity}")

        intensity_choice = input("Enter intensity choice: ")
        intensity_choice = int(intensity_choice)

        if 1 <= intensity_choice <= len(intensities):
            selected_intensity = intensities[intensity_choice - 1]
            selected_other = select_other(selected_item, selected_intensity)
        else:
            print("Invalid intensity choice!")
else:
    print("Invalid choice!")

def select_ecg():
    ecg_performed = input("Was an ECG performed? (Write 1 for Yes or 2 for No): ")
    if ecg_performed == "1":
        ecg_result = input("ECG Result (Write 1 for Normal or 2 for Abnormal): ")
        if ecg_result == "1":
            ecg_output = "An ECG was performed and a Normal Sinus Rhythm was found."
            return ecg_output
        elif ecg_result == "2":
            abnormality = input("Abnormality Type (Write 1 for Arrhythmias, 2 for ST-Segment and T-Wave Abnormalities, 3 for Hypertrophy, 4 for Myocardial Infarction, or 5 for Other): ")
            if abnormality == "1":
                arrhythmia_options = ["Atrial Fibrillation", "Atrial Flutter", "Ventricular Tachycardia", "Ventricular Fibrillation", "Bradycardia", "Heart Block"]
                print("Select Arrhythmia Type:")
                for i, option in enumerate(arrhythmia_options, start=1):
                    print(f"{i}. {option}")

                arrhythmia_choice = input("Enter your choice: ")
                if arrhythmia_choice.isdigit() and 1 <= int(arrhythmia_choice) <= len(arrhythmia_options):
                    arrhythmia_type = arrhythmia_options[int(arrhythmia_choice) - 1]
                    print("Select the degree:")
                    print("1. Mild")
                    print("2. Medium")
                    print("3. Severe")
                    arrhythmia_degree = input("Enter your choice: ")
                    if arrhythmia_degree == "1":
                        arrhythmia_degree = "mild"
                    elif arrhythmia_degree == "2":
                        arrhythmia_degree = "medium"
                    elif arrhythmia_degree == "3":
                        arrhythmia_degree = "severe"
                    else:
                        print("Invalid degree!")
                        return None
                    ecg_output = f"An ECG was performed and it was found abnormal. Specifically, {arrhythmia_type} of {arrhythmia_degree} degree was found."
                    return ecg_output
                else:
                    print("Invalid choice!")
            elif abnormality == "2":
                st_tw_options = ["ST-Segment Elevation", "ST-Segment Depression", "T-Wave Inversion"]
                print("Select ST-Segment and T-Wave Abnormality:")
                for i, option in enumerate(st_tw_options, start=1):
                    print(f"{i}. {option}")

                st_tw_choice = input("Enter your choice: ")
                if st_tw_choice.isdigit() and 1 <= int(st_tw_choice) <= len(st_tw_options):
                    st_tw_abnormality = st_tw_options[int(st_tw_choice) - 1]
                    print("Select the degree:")
                    print("1. Mild")
                    print("2. Medium")
                    print("3. Severe")
                    st_tw_degree = input("Enter your choice: ")
                    if st_tw_degree == "1":
                        st_tw_degree = "mild"
                    elif st_tw_degree == "2":
                        st_tw_degree = "medium"
                    elif st_tw_degree == "3":
                        st_tw_degree = "severe"
                    else:
                        print("Invalid degree!")
                        return None
                    ecg_output = f"An ECG was performed and it was found abnormal. Specifically, {st_tw_abnormality} of {st_tw_degree} degree was found."
                    return ecg_output
                else:
                    print("Invalid choice!")
            elif abnormality == "3":
                hypertrophy_options = ["Left Ventricular Hypertrophy", "Right Ventricular Hypertrophy", "Atrial Enlargement"]
                print("Select Hypertrophy Type:")
                for i, option in enumerate(hypertrophy_options, start=1):
                    print(f"{i}. {option}")

                hypertrophy_choice = input("Enter your choice: ")
                if hypertrophy_choice.isdigit() and 1 <= int(hypertrophy_choice) <= len(hypertrophy_options):
                    hypertrophy_type = hypertrophy_options[int(hypertrophy_choice) - 1]
                    print("Select the degree:")
                    print("1. Mild")
                    print("2. Medium")
                    print("3. Severe")
                    hypertrophy_degree = input("Enter your choice: ")
                    if hypertrophy_degree == "1":
                        hypertrophy_degree = "mild"
                    elif hypertrophy_degree == "2":
                        hypertrophy_degree = "medium"
                    elif hypertrophy_degree == "3":
                        hypertrophy_degree = "severe"
                    else:
                        print("Invalid degree!")
                        return None
                    ecg_output = f"An ECG was performed and it was found abnormal. Specifically, {hypertrophy_type} of {hypertrophy_degree} degree was found."
                    return ecg_output
                else:
                    print("Invalid choice!")
            elif abnormality == "4":
                ecg_output = "An ECG was performed and it was found abnormal. Specifically, traces of Myocardial Infarction were found."
                return ecg_output
            elif abnormality == "5":
                custom_findings = input("Enter your own findings: ")
                ecg_output = f"An ECG was performed and it was found abnormal. Specifically, {custom_findings}."
                return ecg_output
            else:
                print("Invalid abnormality type!")
        else:
            print("Invalid ECG result!")
    elif ecg_performed == "2":
        ecg_output = "No ECG was performed."
        return ecg_output
    else:
        print("Invalid choice!")

ecg_output = select_ecg()

def select_echocardiogram():
    echocardiogram_performed = input("Was an echocardiogram performed? (Write 1 for Yes or 2 for No): ")
    if echocardiogram_performed == "1":
        echocardiogram_result = input("Echocardiogram Result (Write 1 for Normal or 2 for Abnormal): ")
        if echocardiogram_result == "1":
            echocardiogram_output = "An echocardiogram was performed, and the results indicate a normal heart structure and function."
            return echocardiogram_output
        elif echocardiogram_result == "2":
            abnormality = input("Abnormality Type (Write 1 for Valve Disorder, 2 for Congenital Heart Defect, 3 for Cardiomyopathy, 4 for Coronary Artery Disease, 5 for Pericardial Disease, or 6 for Other): ")
            if abnormality == "1":
                valve_disorder_options = ["Valve Stenosis", "Valve Regurgitation", "Valve Prolapse"]
                print("Select Valve Disorder Type:")
                for i, option in enumerate(valve_disorder_options, start=1):
                    print(f"{i}. {option}")

                valve_disorder_choice = input("Enter your choice: ")
                if valve_disorder_choice.isdigit() and 1 <= int(valve_disorder_choice) <= len(valve_disorder_options):
                    valve_disorder_type = valve_disorder_options[int(valve_disorder_choice) - 1]
                    valve_disorder_degree = input("Specify the degree of the disorder (1 for mild, 2 for moderate, 3 for severe): ")
                    if valve_disorder_degree == "1":
                        valve_disorder_degree = "mild"
                    elif valve_disorder_degree == "2":
                        valve_disorder_degree = "moderate"
                    elif valve_disorder_degree == "3":
                        valve_disorder_degree = "severe"
                    else:
                        print("Invalid degree of disorder!")
                        return

                    echocardiogram_output = f"An echocardiogram was performed, and it revealed {valve_disorder_degree} {valve_disorder_type}."
                    return echocardiogram_output
                else:
                    print("Invalid choice!")
            elif abnormality == "2":
                congenital_defect_options = ["Atrial Septal Defect", "Ventricular Septal Defect", "Patent Ductus Arteriosus", "Other"]
                print("Select Congenital Heart Defect Type:")
                for i, option in enumerate(congenital_defect_options, start=1):
                    print(f"{i}. {option}")

                congenital_defect_choice = input("Enter your choice: ")
                if congenital_defect_choice.isdigit() and 1 <= int(congenital_defect_choice) <= len(congenital_defect_options):
                    congenital_defect_type = congenital_defect_options[int(congenital_defect_choice) - 1]
                    congenital_defect_degree = input("Specify the degree of the defect (1 for mild, 2 for moderate, 3 for severe): ")
                    if congenital_defect_degree == "1":
                        congenital_defect_degree = "mild"
                    elif congenital_defect_degree == "2":
                        congenital_defect_degree = "moderate"
                    elif congenital_defect_degree == "3":
                        congenital_defect_degree = "severe"
                    else:
                        print("Invalid degree of defect!")
                        return

                    echocardiogram_output = f"An echocardiogram was performed, and it revealed {congenital_defect_degree} {congenital_defect_type}."
                    return echocardiogram_output
                else:
                    print("Invalid choice!")
            elif abnormality == "3":
                cardiomyopathy_options = ["Dilated Cardiomyopathy", "Hypertrophic Cardiomyopathy", "Restrictive Cardiomyopathy"]
                print("Select Cardiomyopathy Type:")
                for i, option in enumerate(cardiomyopathy_options, start=1):
                    print(f"{i}. {option}")

                cardiomyopathy_choice = input("Enter your choice: ")
                if cardiomyopathy_choice.isdigit() and 1 <= int(cardiomyopathy_choice) <= len(cardiomyopathy_options):
                    cardiomyopathy_type = cardiomyopathy_options[int(cardiomyopathy_choice) - 1]
                    cardiomyopathy_degree = input("Specify the degree of the cardiomyopathy (1 for mild, 2 for moderate, 3 for severe): ")
                    if cardiomyopathy_degree == "1":
                        cardiomyopathy_degree = "mild"
                    elif cardiomyopathy_degree == "2":
                        cardiomyopathy_degree = "moderate"
                    elif cardiomyopathy_degree == "3":
                        cardiomyopathy_degree = "severe"
                    else:
                        print("Invalid degree of cardiomyopathy!")
                        return

                    echocardiogram_output = f"An echocardiogram was performed, and it revealed {cardiomyopathy_degree} {cardiomyopathy_type}."
                    return echocardiogram_output
                else:
                    print("Invalid choice!")
            elif abnormality == "4":
                echocardiogram_output = "An echocardiogram was performed, and it revealed abnormalities indicative of coronary artery disease."
                return echocardiogram_output
            elif abnormality == "5":
                pericardial_disease_options = ["Pericarditis", "Pericardial Effusion", "Constrictive Pericarditis"]
                print("Select Pericardial Disease Type:")
                for i, option in enumerate(pericardial_disease_options, start=1):
                    print(f"{i}. {option}")

                pericardial_disease_choice = input("Enter your choice: ")
                if pericardial_disease_choice.isdigit() and 1 <= int(pericardial_disease_choice) <= len(pericardial_disease_options):
                    pericardial_disease_type = pericardial_disease_options[int(pericardial_disease_choice) - 1]
                    pericardial_disease_degree = input("Specify the degree of the pericardial disease (1 for mild, 2 for moderate, 3 for severe): ")
                    if pericardial_disease_degree == "1":
                        pericardial_disease_degree = "mild"
                    elif pericardial_disease_degree == "2":
                        pericardial_disease_degree = "moderate"
                    elif pericardial_disease_degree == "3":
                        pericardial_disease_degree = "severe"
                    else:
                        print("Invalid degree of pericardial disease!")
                        return

                    echocardiogram_output = f"An echocardiogram was performed, and it revealed {pericardial_disease_degree} {pericardial_disease_type}."
                    return echocardiogram_output
                else:
                    print("Invalid choice!")
            elif abnormality == "6":
                custom_findings = input("Enter your own findings: ")
                echocardiogram_output = f"An echocardiogram was performed, and it revealed the following abnormality: {custom_findings}."
                return echocardiogram_output
            else:
                print("Invalid abnormality type!")
        else:
            print("Invalid echocardiogram result!")
    elif echocardiogram_performed == "2":
        echocardiogram_output = "No echocardiogram was performed."
        return echocardiogram_output
    else:
        print("Invalid choice!")

def select_echocardiogram():
    echocardiogram_performed = input("Was an echocardiogram performed? (Write 1 for Yes or 2 for No): ")
    if echocardiogram_performed == "1":
        echocardiogram_result = input("Echocardiogram Result (Write 1 for Normal or 2 for Abnormal): ")
        if echocardiogram_result == "1":
            echocardiogram_output = "An echocardiogram was performed, and the results indicate a normal heart structure and function."
            return echocardiogram_output
        elif echocardiogram_result == "2":
            abnormality = input("Abnormality Type (Write 1 for Valve Disorder, 2 for Congenital Heart Defect, 3 for Cardiomyopathy, 4 for Coronary Artery Disease, 5 for Pericardial Disease, or 6 for Other): ")
            if abnormality == "1":
                valve_disorder_options = ["Valve Stenosis", "Valve Regurgitation", "Valve Prolapse"]
                print("Select Valve Disorder Type:")
                for i, option in enumerate(valve_disorder_options, start=1):
                    print(f"{i}. {option}")

                valve_disorder_choice = input("Enter your choice: ")
                if valve_disorder_choice.isdigit() and 1 <= int(valve_disorder_choice) <= len(valve_disorder_options):
                    valve_disorder_type = valve_disorder_options[int(valve_disorder_choice) - 1]
                    valve_disorder_degree = input("Specify the degree of the disorder (1 for mild, 2 for moderate, 3 for severe): ")
                    if valve_disorder_degree == "1":
                        valve_disorder_degree = "mild"
                    elif valve_disorder_degree == "2":
                        valve_disorder_degree = "moderate"
                    elif valve_disorder_degree == "3":
                        valve_disorder_degree = "severe"
                    else:
                        print("Invalid degree of disorder!")
                        return

                    echocardiogram_output = f"An echocardiogram was performed, and it revealed {valve_disorder_degree} {valve_disorder_type}."
                    return echocardiogram_output
                else:
                    print("Invalid choice!")
            elif abnormality == "2":
                congenital_defect_options = ["Atrial Septal Defect", "Ventricular Septal Defect", "Patent Ductus Arteriosus", "Other"]
                print("Select Congenital Heart Defect Type:")
                for i, option in enumerate(congenital_defect_options, start=1):
                    print(f"{i}. {option}")

                congenital_defect_choice = input("Enter your choice: ")
                if congenital_defect_choice.isdigit() and 1 <= int(congenital_defect_choice) <= len(congenital_defect_options):
                    congenital_defect_type = congenital_defect_options[int(congenital_defect_choice) - 1]
                    congenital_defect_degree = input("Specify the degree of the defect (1 for mild, 2 for moderate, 3 for severe): ")
                    if congenital_defect_degree == "1":
                        congenital_defect_degree = "mild"
                    elif congenital_defect_degree == "2":
                        congenital_defect_degree = "moderate"
                    elif congenital_defect_degree == "3":
                        congenital_defect_degree = "severe"
                    else:
                        print("Invalid degree of defect!")
                        return

                    echocardiogram_output = f"An echocardiogram was performed, and it revealed {congenital_defect_degree} {congenital_defect_type}."
                    return echocardiogram_output
                else:
                    print("Invalid choice!")
            elif abnormality == "3":
                cardiomyopathy_options = ["Dilated Cardiomyopathy", "Hypertrophic Cardiomyopathy", "Restrictive Cardiomyopathy"]
                print("Select Cardiomyopathy Type:")
                for i, option in enumerate(cardiomyopathy_options, start=1):
                    print(f"{i}. {option}")

                cardiomyopathy_choice = input("Enter your choice: ")
                if cardiomyopathy_choice.isdigit() and 1 <= int(cardiomyopathy_choice) <= len(cardiomyopathy_options):
                    cardiomyopathy_type = cardiomyopathy_options[int(cardiomyopathy_choice) - 1]
                    cardiomyopathy_degree = input("Specify the degree of the cardiomyopathy (1 for mild, 2 for moderate, 3 for severe): ")
                    if cardiomyopathy_degree == "1":
                        cardiomyopathy_degree = "mild"
                    elif cardiomyopathy_degree == "2":
                        cardiomyopathy_degree = "moderate"
                    elif cardiomyopathy_degree == "3":
                        cardiomyopathy_degree = "severe"
                    else:
                        print("Invalid degree of cardiomyopathy!")
                        return

                    echocardiogram_output = f"An echocardiogram was performed, and it revealed {cardiomyopathy_degree} {cardiomyopathy_type}."
                    return echocardiogram_output
                else:
                    print("Invalid choice!")
            elif abnormality == "4":
                echocardiogram_output = "An echocardiogram was performed, and it revealed abnormalities indicative of coronary artery disease."
                return echocardiogram_output
            elif abnormality == "5":
                pericardial_disease_options = ["Pericarditis", "Pericardial Effusion", "Constrictive Pericarditis"]
                print("Select Pericardial Disease Type:")
                for i, option in enumerate(pericardial_disease_options, start=1):
                    print(f"{i}. {option}")

                pericardial_disease_choice = input("Enter your choice: ")
                if pericardial_disease_choice.isdigit() and 1 <= int(pericardial_disease_choice) <= len(pericardial_disease_options):
                    pericardial_disease_type = pericardial_disease_options[int(pericardial_disease_choice) - 1]
                    pericardial_disease_degree = input("Specify the degree of the pericardial disease (1 for mild, 2 for moderate, 3 for severe): ")
                    if pericardial_disease_degree == "1":
                        pericardial_disease_degree = "mild"
                    elif pericardial_disease_degree == "2":
                        pericardial_disease_degree = "moderate"
                    elif pericardial_disease_degree == "3":
                        pericardial_disease_degree = "severe"
                    else:
                        print("Invalid degree of pericardial disease!")
                        return

                    echocardiogram_output = f"An echocardiogram was performed, and it revealed {pericardial_disease_degree} {pericardial_disease_type}."
                    return echocardiogram_output
                else:
                    print("Invalid choice!")
            elif abnormality == "6":
                custom_findings = input("Enter your own findings: ")
                echocardiogram_output = f"An echocardiogram was performed, and it revealed the following abnormality: {custom_findings}."
                return echocardiogram_output
            else:
                print("Invalid abnormality type!")
        else:
            print("Invalid echocardiogram result!")
    elif echocardiogram_performed == "2":
        echocardiogram_output = "No echocardiogram was performed."
        return echocardiogram_output
    else:
        print("Invalid choice!")

echocardiogram_output = select_echocardiogram()


print('Gathering the information and building the report. Please be patient.')



Enter patient ID code: 12345
Enter the patient'age: 45
Select the patient's gender:
1. Female
2. Male
3. Trans-female
4. Trans-male
Enter your choice: 2
Selected gender: Male
Enter systolic BP: 120
Enter diastolic BP: 80
Select ausculted breath sound:
1. Crackles (rales)
2. Wheezes
3. Ronchi
4. Normal
Enter your choice: 1
Select intensity:
1. mild
2. medium
3. intense
Enter intensity choice: 1
Select ausculted heart sounds:
1. Murmurs
2. Gallops
3. Normal
Enter your choice: 1
Select intensity:
1. mild
2. medium
3. intense
Enter intensity choice: 1
Select other ausculted sounds:
1. friction rubs
2. Gallops
3. Normal
Enter your choice: 3
Was an ECG performed? (Write 1 for Yes or 2 for No): 1
ECG Result (Write 1 for Normal or 2 for Abnormal): 2
Abnormality Type (Write 1 for Arrhythmias, 2 for ST-Segment and T-Wave Abnormalities, 3 for Hypertrophy, 4 for Myocardial Infarction, or 5 for Other): 1
Select Arrhythmia Type:
1. Atrial Fibrillation
2. Atrial Flutter
3. Ventricular Tachycardia
4. 

## Medical Report Generation

The `generate_medical_report` function processes the concatenated patient information (patient ID, age, gender, blood pressure, auscultated breath sounds, heart sounds, other findings, ECG results, and echocardiogram results) and uses the OpenAI API for text generation. Here's how it works:

1. **Concatenation of Patient Information:**
   - The function constructs a string `doctor_notes` by concatenating patient ID, age, gender, blood pressure, auscultated breath sounds, heart sounds, other findings, ECG results, and echocardiogram results.

2. **OpenAI API Integration:**
   - The function utilizes the OpenAI API to generate a medical report.
   - It sets up the starting message with the concatenated `doctor_notes`, providing initial context for the AI model.
  
3. **Text Generation:**
   - Using the `text-davinci-003` engine, the function generates a medical report based on the provided context and the concatenated patient information.
   - The `max_tokens` parameter limits the response length to ensure a concise report.
   - The `temperature` parameter controls the creativity of the generated text; lower values (like 0.1) result in more focused and deterministic output.
   - The `n` parameter specifies the number of alternative completions to generate, with `n=1` indicating a single response.
   - The `timeout` parameter sets the maximum time to wait for the API response.

4. **Returned Medical Report:**
   - The function extracts the generated medical report from the API response and returns it to later be printed.

In [4]:
# set your open AI API key

API_key = ""

os.environ["OPENAI_API_KEY"] = API_key


client = openai.Client()


In [6]:
%%time

doctor_notes = str(patient_id) + str(age) + str(gender) + str(BP_reg) + str(selected_breath) + str(selected_heart) + str(selected_other) + str(ecg_output) + str(echocardiogram_output)




def generate_medical_report(doctor_notes):

  # Define the starting message with the doctor notes
  starting_message = f"Doctor Notes: {doctor_notes}\nMedical Report:"

  # Generate the medical report using ChatGPT
  response = client.completions.create(
    model="text-davinci-003",
    prompt=starting_message,
    max_tokens=500,
    temperature=0.1,
    n=1,
    stop=None,
    timeout=15
  )

  # Extract the generated medical report from the response
  generated_text = response.choices[0].text.strip()

  return generated_text

# Generate the medical report
medical_report = generate_medical_report(doctor_notes)

# Print the generated medical report
print(medical_report)

Patient 12345 is a 45-year-old male with a registered blood pressure of 120.0/80.0 mmHg. On auscultation, crackles (rales) were heard with a mild intensity, and murmurs were heard with a mild intensity. No other ausculted sounds were present. An ECG revealed bradycardia of mild degree, and an echocardiogram revealed mild hypertrophic cardiomyopathy. Further evaluation and treatment is recommended.
CPU times: user 41 ms, sys: 1.93 ms, total: 43 ms
Wall time: 2.56 s


In [7]:
def count_words(string):
    word_list = string.split()  # Split the string by whitespace to obtain individual words
    word_count = len(word_list)  # Count the number of words in the list
    return word_count

count_words(medical_report)

58

## Text Quality Assessment with BERT-based Model

The `calculate_text_quality_BIOBERT` function assesses the quality of a given text using the Bio_ClinicalBERT model. Here's an overview of how the function operates:

1. **Model Selection and Initialization:**
   - The function uses the 'emilyalsentzer/Bio_ClinicalBERT' pre-trained model for text classification.
   - It also loads the corresponding tokenizer for text preprocessing.

2. **Text Preprocessing:**
   - The input text is tokenized and preprocessed using the loaded tokenizer.
   - Padding and truncation are applied to ensure consistent input format for the model.
  
3. **Prediction and Quality Score:**
   - The preprocessed text is fed into the Bio_ClinicalBERT model for sequence classification.
   - The function computes predictions using the model's logits and selects the class with the highest probability as the predicted class.
   - Predicted class values are mapped to quality scores: Class 0 -> 2, Class 1 -> 4, Class 2 -> 6, Class 3 -> 8, Class 4 -> 10.
   - The resulting quality score represents the assessed quality of the input text, ranging from 2 to 10.

4. **Example Usage:**
   - The function demonstrates its functionality by calculating the text quality score for a given medical report (`medical_report` variable).
   - The calculated score is printed to provide insight into the quality assessment results.



In [8]:
%%time




def calculate_text_quality_BIOBERT(text):
    model_name = 'emilyalsentzer/Bio_ClinicalBERT'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name)

    # Preprocess the text
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Make predictions
    with torch.no_grad():
        outputs = model(**encoded_input)
        predicted_class = torch.argmax(outputs.logits).item()

    # Mapping predicted class to a quality score
    class_scores = {0: 2, 1: 4, 2: 6, 3: 8, 4: 10}
    text_quality_score = class_scores[predicted_class]

    return text_quality_score


score_1 = calculate_text_quality_BIOBERT(medical_report)
print(score_1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


4
CPU times: user 3.01 s, sys: 1.6 s, total: 4.61 s
Wall time: 8.07 s


## Text Quality Assessment with XLM-Roberta Model

The `calculate_text_quality_ROBERTA` function evaluates the quality of a given text using the XLM-Roberta model. Here's an overview of how the function operates:

1. **Model Selection and Initialization:**
   - The function utilizes the 'xlm-roberta-base' pre-trained model for sequence classification.
   - It also loads the corresponding tokenizer for text preprocessing.

2. **Text Preprocessing:**
   - The input text is tokenized and preprocessed using the loaded tokenizer.
   - Padding and truncation are applied to ensure consistent input format for the model.
  
3. **Prediction and Quality Score:**
   - The preprocessed text is fed into the XLM-Roberta model for sequence classification.
   - The function computes predictions using the model's logits and selects the class with the highest probability as the predicted class.
   - Predicted class values are mapped to quality scores: Class 0 -> 2, Class 1 -> 4, Class 2 -> 6, Class 3 -> 8, Class 4 -> 10.
   - The resulting quality score represents the assessed quality of the input text, ranging from 2 to 10.

4. **Example Usage:**
   - The function demonstrates its functionality by calculating the text quality score for a given medical report (`medical_report` variable).
   - The calculated score is printed to provide insight into the quality assessment results.

In [12]:
%%time
'''

def calculate_text_quality_ROBERTA(text):
    model_name = 'xlm-roberta-base'
    tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
    model = XLMRobertaForSequenceClassification.from_pretrained(model_name)

    # Preprocess the text
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Make predictions
    with torch.no_grad():
        outputs = model(**encoded_input)
        predicted_class = torch.argmax(outputs.logits).item()

    # Mapping predicted class to a quality score
    class_scores = {0: 2, 1: 4, 2: 6, 3: 8, 4: 10}
    text_quality_score = class_scores[predicted_class]

    return text_quality_score


score_2 = calculate_text_quality_ROBERTA(medical_report)
print(score_2) '''

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.3 µs


"\n\ndef calculate_text_quality_ROBERTA(text):\n    model_name = 'xlm-roberta-base'\n    tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)\n    model = XLMRobertaForSequenceClassification.from_pretrained(model_name)\n\n    # Preprocess the text\n    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')\n\n    # Make predictions\n    with torch.no_grad():\n        outputs = model(**encoded_input)\n        predicted_class = torch.argmax(outputs.logits).item()\n\n    # Mapping predicted class to a quality score\n    class_scores = {0: 2, 1: 4, 2: 6, 3: 8, 4: 10}\n    text_quality_score = class_scores[predicted_class]\n\n    return text_quality_score\n\n\nscore_2 = calculate_text_quality_ROBERTA(medical_report)\nprint(score_2) "

## Text Quality Assessment with ChatGPT 3.4

The `calculate_text_quality_GPT` function assesses the quality of a given text using ChatGPT 3.4. Here's how the function works:

1. **Prompt Definition:**
   - The function constructs a prompt asking the model to assess the quality of the provided text.
   - The input text (`text` variable) is included in the prompt for evaluation.

2. **Generation of Quality Score:**
   - The function uses the OpenAI API to generate a response from ChatGPT 3.4.
   - The response is limited to a single token, representing the quality score of the provided text.
   - The model determines the response length based on the given prompt and context.
   - The extracted quality score represents the assessed quality of the input text.

3. **Example Usage:**
   - The function showcases its functionality by calculating the text quality score for a given medical report (`medical_report` variable).
   - The calculated quality score is printed to provide insight into the assessment results.


In [13]:
%%time
def calculate_text_quality_GPT(text):


  prompt = f"Assess the quality of the following text:\n\n{text}\n\nQuality score:"

  response = client.completions.create(
     model="text-davinci-003",
     prompt=prompt,
     max_tokens=1,
     n=1,
     stop=None,
     temperature=0
  )

  quality_score = int(response.choices[0].text.strip())

  return quality_score



# Example usage

quality_score = calculate_text_quality_GPT(medical_report)
print(quality_score)

8
CPU times: user 17.7 ms, sys: 1.69 ms, total: 19.4 ms
Wall time: 469 ms
